In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
import keras
np.random.seed(2)
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.layers import Dropout, Conv2D, MaxPooling2D, Flatten, Dense
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, Adadelta

sns.set(style='white', context='notebook', palette='deep')

Using TensorFlow backend.


In [2]:
# Load the data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print("Shape of train data", train_df.shape)
print("Shape of test data", test_df.shape)

Shape of train data (42000, 785)
Shape of test data (28000, 784)


In [3]:
# seperating the label
train_df=np.array(train_df)
test_df=np.array(test_df)
x = train_df[:,1:]
y = train_df[:,0]

In [4]:
#splitting the data
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.33, shuffle = True)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(28140, 784) (13860, 784) (28140,) (13860,)


In [5]:
#normalising the data and converting the labels to one hot encoded vectors
batch_size = 128
num_classes = 10
epochs = 1
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    test_df=test_df.reshape(test_df.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    test_df=test_df.reshape(test_df.shape[0], img_rows, img_cols,1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', test_df.shape)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)

x_train shape: (28140, 28, 28, 1)
y_train shape: (28140,)
x_test shape: (28000, 28, 28, 1)


In [6]:
# Conv2D Model
model = Sequential()
model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(256, (3, 3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(256, (3, 3),padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3),padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
#model.summary()
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


In [8]:
datagen.fit(x_train)
history = model.fit(x_train, y_train, batch_size, epochs = epochs, verbose = 1, validation_data = (x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 28140 samples, validate on 13860 samples
Epoch 1/1
28140/28140 [==============================] - 635s 23ms/step - loss: 14.0352 - acc: 0.1058 - val_loss: 14.5784 - val_acc: 0.0955


In [ ]:
pred = model.predict(test_df)

In [ ]:
result = np.argmax(pred,axis = 1)
np.savetxt('result.csv',
          np.c_[range(1,len(test_df) + 1),result],
          delimiter=',',
            header='ImageId,Label',
           comments='',
          fmt='%d')